In [1]:
import random
import pandas as pd
from data_vis.card_logic import CardLogic 
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
class NeuralNetwork:
    def __init__(self, input_size=98 , output_size=1):
        self.input_size = input_size  # Save input_size as an instance variable
        self.output_size = output_size  # Save output_size as an instance variable
        self.model = self.build_model()
    def build_model(self):
        model = models.Sequential()
        model.add(layers.Dense(64, input_dim=98, activation='relu'))
        model.add(layers.Dense(32, activation='relu'))
        model.add(layers.Dense(self.output_size, activation='linear'))
        model.compile(optimizer='adam', loss='mse')  # Mean Squared Error for regression task
        return model


    def train(self, state, target):
        self.model.fit(state, target, epochs=1, verbose=0)

    def predict(self, state):
        return self.model.predict(state)
class ClashRoyaleGame:
    def __init__(self):
        self.player1_tower_health = 1000
        self.player2_tower_health = 1000
        self.player1_deck = ['Zap', 'The Log', 'Fireball', 'Arrows', 'Mega Knight', 'Skeleton Armys', 'Valkyrie', 'Wizard']
        self.player2_deck = self.get_player2_cards()

        # Initialize player hands
        self.player1_hand_size = 4
        self.player1_hand = self.draw_cards(self.player1_deck, self.player1_hand_size)

        self.player2_hand_size = 4  # Player 2 starts with 4 cards in hand
        self.player2_hand = self.draw_cards(self.player2_deck, self.player2_hand_size)

        # Instantiate the CardLogic class
        self.card_logic = CardLogic()

    def get_player2_cards(self):
        # Read cards from CSV file for Player 2
        player2_card_data = pd.read_csv(r'C:\Users\Cdog1\OneDrive\Documents\MEM 680T\Final Visuals\Data_vis\src\data_vis\datapath\500266047.csv')  # Replace with your actual CSV file path
        player2_deck = player2_card_data['team.card1.name'].tolist()

        # Ensure Player 2 starts with 8 cards from the CSV file
        starting_hand_size = 8
        player2_hand = self.draw_cards(player2_deck, starting_hand_size)

        return player2_deck

    def draw_cards(self, deck, num_cards):
        # Draw a specified number of cards from the deck
        drawn_cards = random.sample(deck, num_cards)
        return drawn_cards

    def display_game_state(self, player_hand):
        print(f"Player 1 Tower Health: {self.player1_tower_health}")
        print(f"Player 2 Tower Health: {self.player2_tower_health}")

        if player_hand == self.player1_hand:
            print(f"Player 1 Hand: {player_hand}")
        elif player_hand == self.player2_hand:
            print(f"Player 2 Hand: {player_hand}")

    def player_turn(self, player_deck, player_hand):
        
        selected_card = input("Choose a card to deploy (type 'exit' to end the game): ")

        if selected_card.lower() == 'exit':
            print("Game ended by player.")
            exit()

        if selected_card not in player_hand:
            print("Invalid card selection. Try again.")
            return self.player_turn(player_deck, player_hand)

        # Call the corresponding card logic function based on the selected card
    
        temp_selected_card = selected_card.replace(" ", "_").replace(".", "").replace("-", "_").lower()
        card_logic_function = getattr(self.card_logic, f"{temp_selected_card}_logic", None)
        if card_logic_function:
            card_result = card_logic_function()
            if isinstance(card_result, dict):
                damage = card_result.get('damage', 0)  # Assuming default damage is 0 if not present in the dictionary
                # Handle any additional logic or effects here...
                print("Amount of Damage:", damage)
            else:
                damage = card_result
                print("Amount of Damage:", damage)
            return selected_card, damage
        else:
            print("Invalid card logic. Try again.")
            return self.player_turn(player_deck, player_hand)


    def update_player_hand(self, player_deck, player_hand, used_card):
        # Move the used card to the back of the deck
        player_deck.remove(used_card)
        player_deck.append(used_card)

        # Draw a new card from the deck that is not in the current hand
        new_card = random.choice([card for card in player_deck if card not in player_hand])
        player_hand.remove(used_card)
        player_hand.append(new_card)

    def play(self):
        while self.player1_tower_health > 0 and self.player2_tower_health > 0:
            print("\n=== Player 1's Turn ===")
            self.display_game_state(self.player1_hand)  # Pass the current player's hand to display_game_state
            player1_card, player1_damage = self.player_turn(self.player1_deck, self.player1_hand)
            self.player2_tower_health -= player1_damage
            self.update_player_hand(self.player1_deck, self.player1_hand, player1_card)

            if self.player2_tower_health <= 0:
                print("Player 1 wins!")
                break

            print("\n=== Player 2's Turn ===")
            self.display_game_state(self.player2_hand)  # Pass the current player's hand to display_game_state
            player2_card, player2_damage = self.player_turn(self.player2_deck, self.player2_hand)
            self.player1_tower_health -= player2_damage
            self.update_player_hand(self.player2_deck, self.player2_hand, player2_card)

            if self.player1_tower_health <= 0:
                print("Player 2 wins!")
                break

    def test_card_logic(self):
        all_cards = self.player1_deck + self.player2_deck

        for card in all_cards:
            card_name = card.lower().replace(" ", "_")
            card_logic_function = getattr(self.card_logic, f"{card_name}_logic", None)

            if card_logic_function:
                try:
                    card_result = card_logic_function()
                    if not isinstance(card_result, (int, dict)):
                        print(f"Error: Invalid result for {card}")
                except Exception as e:
                    print(f"Error: Exception in {card} logic - {e}")
            else:
                print(f"Error: Missing logic for {card}")
    def simulate_game_for_network(self, network):
        while self.player1_tower_health > 0 and self.player2_tower_health > 0:
            print("\n=== Player 1's Turn ===")
            self.display_game_state(self.player1_hand)
            player1_card, player1_damage = self.choose_card_for_network(network, self.player1_deck, self.player1_hand)
            self.player2_tower_health -= player1_damage
            self.update_player_hand(self.player1_deck, self.player1_hand, player1_card)

            if self.player2_tower_health <= 0:
                print("Player 1 wins!")
                break

            print("\n=== Player 2's Turn ===")
            self.display_game_state(self.player2_hand)
            player2_card, player2_damage = self.choose_card_for_network(network, self.player2_deck, self.player2_hand)
            self.player1_tower_health -= player2_damage
            self.update_player_hand(self.player2_deck, self.player2_hand, player2_card)

            if self.player1_tower_health <= 0:
                print("Player 2 wins!")
                break

    def choose_card_for_network(self, network, player_deck, player_hand, tower_health):
        # Create a list to store predicted damages for each card in the hand
        predicted_damages = []

        if not player_hand:
            print("Player hand is empty. No card to choose.")
            return None, 0  # or some default values

        for card in player_hand:
            state = self.get_state_representation(player_deck, player_hand, tower_health, available_cards=player_hand)
            predicted_damage = int(network.predict(state)[0][0])  # Convert the predicted damage to an integer
            predicted_damages.append(predicted_damage)

        # Choose the card with the highest predicted damage
        selected_card_index = np.argmax(predicted_damages)
        selected_card = player_hand[selected_card_index]

        # Call the corresponding card logic function based on the selected card
        temp_selected_card = selected_card.replace(" ", "_").replace(".", "").replace("-", "_").lower()
        card_logic_function = getattr(self.card_logic, f"{temp_selected_card}_logic", None)
        if card_logic_function:
            card_result = card_logic_function()
            if isinstance(card_result, dict):
                damage = int(card_result.get('damage', 0))  # Convert the damage to an integer
                print("Amount of Damage:", damage)
            else:
                damage = int(card_result)
                print("Amount of Damage:", damage)
            return selected_card, damage
        else:
            print("Invalid card logic. Try again.")
            return self.choose_card_for_network(network, player_deck, player_hand, tower_health)


    def get_state_representation(self, player_deck, player_hand, tower_health, opponent_tower_health=None, available_cards=None):
        # Convert categorical data (e.g., card names) to numerical representations
        player_hand_encoding = [self.card_encoding(card, available_cards) for card in player_hand]
        player_deck_encoding = [self.card_encoding(card, available_cards) for card in player_deck]

        # Set default values if not provided
        opponent_tower_health = opponent_tower_health if opponent_tower_health is not None else 1000
        available_cards = available_cards if available_cards is not None else player_hand  # Use player_hand as default

        # Combine all features into a single array
        state = np.concatenate([
            np.concatenate(player_hand_encoding),
            np.concatenate(player_deck_encoding),
            np.array([tower_health, opponent_tower_health])
        ])

        # Add an extra dimension to the state array
        state = np.expand_dims(state, axis=0)

        return state


    def card_encoding(self, card, available_cards=None):
        # Set default value if not provided
        available_cards = available_cards if available_cards is not None else []

        encoding = [1 if card == c else 0 for c in available_cards]
        return encoding

    def train_neural_network(self, network, player_deck, player_hand, tower_health, damage):
        state = self.get_state_representation(player_deck, player_hand, tower_health)
        target = np.array([damage])  # Target value for the neural network

        network.train(state, target)

    def choose_card(self, network, player_deck, player_hand, tower_health, available_cards):
        state = self.get_state_representation(player_deck, player_hand, tower_health, available_cards=available_cards)
        predicted_damage = int(network.predict(state)[0][0])  # Convert the predicted damage to an integer

        # Choose the card with the highest predicted damage
        selected_card_index = np.argmax(predicted_damage)
        selected_card = player_hand[selected_card_index]

        return selected_card, predicted_damage

    def read_all_cards_from_csv(self, csv_path):
        # Read all cards from the CSV file
        card_data = pd.read_csv(csv_path)
        all_cards = card_data['team.card1.name'].tolist()
        return all_cards
    def get_random_subset_of_cards(self, all_cards, subset_size):
        # Get a random subset of cards
        return random.sample(all_cards, subset_size)

    def play_with_neural_network(self, player1_network, player2_network, max_turns=100):
        current_turn = 0

        # Read all cards from the CSV file
        all_cards = self.read_all_cards_from_csv(r'C:\Users\Cdog1\OneDrive\Documents\MEM 680T\Final Visuals\Data_vis\src\data_vis\datapath\500266047.csv')  # Replace with the actual path

        while self.player1_tower_health > 0 and self.player2_tower_health > 0 and current_turn < max_turns:
            # Get a random subset of cards for both players in each game
            available_cards_player1 = self.get_random_subset_of_cards(all_cards, subset_size)
            available_cards_player2 = self.get_random_subset_of_cards(all_cards, subset_size)

            # Player 1's turn
            print("\n=== Player 1's Turn ===")
            self.display_game_state(self.player1_hand)
            player1_card = self.choose_card(player1_network, self.player1_deck, self.player1_hand, self.player1_tower_health, available_cards_player1)
            self.player2_tower_health -= player1_card[1]  # Use the second element of the tuple for damage
            self.update_player_hand(self.player1_deck, self.player1_hand, player1_card[0])
            self.train_neural_network(player1_network, self.player1_deck, self.player1_hand, self.player1_tower_health, player1_card[1])

            # Player 2's turn
            print("\n=== Player 2's Turn ===")
            self.display_game_state(self.player2_hand)
            player2_card = self.choose_card(player2_network, self.player2_deck, self.player2_hand, self.player2_tower_health, available_cards_player2)
            self.player1_tower_health -= player2_card[1]  # Use the second element of the tuple for damage
            self.update_player_hand(self.player2_deck, self.player2_hand, player2_card[0])
            self.train_neural_network(player2_network, self.player2_deck, self.player2_hand, self.player2_tower_health, player2_card[1])

            if self.player1_tower_health <= 0:
                print("Player 2 wins!")
                break

            current_turn += 2  # Increment turns for both players

        if current_turn >= max_turns:
            print("Game reached the maximum turn limit.")

            
if __name__ == "__main__":
    N = 8  # Number of possible cards in the hand
    M = 50  # Number of possible cards in the deck
    input_size = N + M + 2  # Player's hand + Player's deck + Tower health + Opponent tower health
    output_size = 1  # Assuming a regression task predicting damage
    subset_size=8
    player1_network = NeuralNetwork(input_size, output_size)
    player2_network = NeuralNetwork(input_size, output_size)
    num_games = 1000

    for game_num in range(num_games):
        game = ClashRoyaleGame()

        # Train neural networks with the simulated game
        game.play_with_neural_network(player1_network, player2_network)

        # Optionally, print or log information about the game and neural network's learning progress
        print(f"Game {game_num + 1} completed.")

    # Evaluate the performance of the neural networks
    num_evaluation_games = 100  # Set the number of games for evaluation

    player1_wins = 0
    player2_wins = 0

    for eval_game_num in range(num_evaluation_games):
        eval_game = ClashRoyaleGame()

        # Play the game using the trained neural networks without further training
        eval_game.play_with_neural_network(player1_network, player2_network)

        # Check the winner and update win counts
        if eval_game.player1_tower_health <= 0:
            player1_wins += 1
        elif eval_game.player2_tower_health <= 0:
            player2_wins += 1

    # Calculate win rates
    player1_win_rate = player1_wins / num_evaluation_games
    player2_win_rate = player2_wins / num_evaluation_games

    print(f"Player 1 Win Rate: {player1_win_rate * 100:.2f}%")
    print(f"Player 2 Win Rate: {player2_win_rate * 100:.2f}%")

    # Save the trained models if needed
    player1_network.model.save("player1_model.h5")
    player2_network.model.save("player2_model.h5")







=== Player 1's Turn ===
Player 1 Tower Health: 1000
Player 2 Tower Health: 1000
Player 1 Hand: ['Fireball', 'Mega Knight', 'The Log', 'Arrows']
1/1 [==============================] - 0s 132ms/step


ValueError: in user code:

    File "c:\Users\Cdog1\anaconda3\envs\Final\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Cdog1\anaconda3\envs\Final\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Cdog1\anaconda3\envs\Final\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Cdog1\anaconda3\envs\Final\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Cdog1\anaconda3\envs\Final\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Cdog1\anaconda3\envs\Final\lib\site-packages\keras\src\engine\input_spec.py", l